In [18]:
# read the text file "NE.population" and store in the list called population
population = list()

# open the text file for reading
filepath = "C:\\Users\\jayde\\Downloads\\"
filename = "NE.population .Copy.txt"
file = open( filepath + filename ,"r")

# while the current line is not empty, read in a new county population
line = file.readline()


while line != "":
    # split the line into two "words": 
    #    word[0]: the county's number
    #    word[1]: the county's population
    words = line.split() 
    county_number = words[0]
    county_population = int(words[1]) # cast the string as type int
    
    # append to population list
    population.append(county_population)
    
    # read next line
    line = file.readline() 

file.close()
print("population = ",population)

population =  [6129, 9998, 11308, 818, 7547, 15740, 36288, 736, 11055, 614, 36691, 3966, 158840, 36970, 20234, 2538, 3812, 517110, 25241, 5713, 5505, 5217, 9182, 2756, 4500, 6489, 6000, 6274, 824, 763, 8852, 632, 5406, 478, 3821, 2044, 3423, 9188, 9124, 8395, 10435, 32237, 5890, 10939, 46102, 6858, 24326, 20780, 647, 6685, 2049, 1311, 1941, 2773, 2099, 5228, 2908, 690, 4959, 6542, 7845, 3145, 10515, 21006, 460, 5469, 16750, 9595, 8701, 7248, 5042, 58607, 4260, 34876, 3225, 3735, 967, 22311, 13665, 539, 2970, 1526, 285407, 31364, 9139, 2057, 8368, 6940, 7266, 8363, 14200, 3152, 2008]


In [19]:
# we are to solve the following task:
# input: a population vector, desired number of districts k
# output: a partition of the populations into k districts (not necessarily connected!) 
#            to minimize the difference between most and least populated districts

import gurobipy as gp
from gurobipy import GRB

In [20]:
n = len(population) # number of counties
k = 3               # desired number of districts

# create model 
m = gp.Model()

# create variables
x = m.addVars(n,k,vtype=GRB.BINARY) # x_ij equals one when county i is assigned to district j
y = m.addVar()                      # the population of least-populated district
z = m.addVar()                      # the population of most-populated district

Academic license - for non-commercial use only - expires 2021-06-22
Using license file C:\Users\jayde\gurobi.lic


In [21]:
# objective is to minimize absolute population deviation
m.setObjective(z-y,GRB.MINIMIZE)

# add constraints saying that each county i is assigned to one district
m.addConstrs( sum(x[i,j] for j in range(k)) == 1 for i in range(n) )

# add constraints saying that each district has population at least y
m.addConstrs( y <= sum( population[i] * x[i,j] for i in range(n) ) for j in range(k) )

# add constraints saying that each district has population at most z
m.addConstrs( sum( population[i] * x[i,j] for i in range(n)) <= z for j in range(k) )

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [22]:
# solve IP model
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 99 rows, 281 columns and 843 nonzeros
Model fingerprint: 0x5af52a33
Variable types: 2 continuous, 279 integer (279 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 652429.00000
Presolve time: 0.00s
Presolved: 99 rows, 281 columns, 843 nonzeros
Variable types: 0 continuous, 281 integer (279 binary)

Root relaxation: objective 0.000000e+00, 89 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    6 652429.000    0.00000   100%     -    0s
H    0     0                    41278.000000    0.00000   100%     -    0s
H    0     0       

In [23]:
# print the absolute population deviation
print("The absolute population deviation is",m.objval,"person(s).")

# retrieve the districts and their populations
districts = [ [i for i in range(n) if x[i,j].x > 0.5] for j in range(k)]
district_populations = [ sum(population[i] for i in district) for district in districts ]

# print district info
for j in range(k):
    print("District",j,"has population",district_populations[j],"and contains counties",districts[j])

The absolute population deviation is 1.0 person(s).
District 0 has population 608780 and contains counties [0, 1, 2, 4, 9, 11, 13, 15, 19, 21, 24, 25, 26, 32, 34, 35, 36, 38, 40, 47, 48, 49, 51, 52, 55, 57, 58, 59, 61, 62, 63, 65, 67, 68, 69, 70, 72, 74, 75, 76, 77, 79, 80, 82, 85, 89, 90]
District 1 has population 608781 and contains counties [3, 7, 16, 17, 20, 23, 28, 29, 31, 42, 44, 45, 50, 53, 54, 56, 64, 81, 91, 92]
District 2 has population 608780 and contains counties [5, 6, 8, 10, 12, 14, 18, 22, 27, 30, 33, 37, 39, 41, 43, 46, 60, 66, 71, 73, 78, 83, 84, 86, 87, 88]
